## 电影推荐系统

> [https://cloud.tencent.com/developer/article/1058235](https://cloud.tencent.com/developer/article/1058235)

> [当推荐系统遇上深度学习](https://blog.csdn.net/somTian/article/details/71516613)

> [鉴于IMDB电影ID，我如何以编程方式获取其海报图像？](https://oomake.com/question/17663)

> [W1_017陈佳豪](https://www.kesci.com/home/project/5a73e21659774204c69f2559) -- 有实现获取海报的代码

> [TMDB电影数据分析](https://www.kesci.com/home/project/5afa3eaf3878b214d9ca9fff)

> [趣味项目 episode 2——IMDB电影数据分析](https://www.kesci.com/home/project/5ad5a5687238515d80b55cba)

> [一个电影推荐系统，毕业设计](https://github.com/JaniceWuo/MovieRecommend)

> []()

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from collections import Counter
import tensorflow as tf

import os
import pickle
import re
from tensorflow.python.ops import math_ops

from matplotlib import pyplot as plt
import seaborn as sns

%matplotlib inline

In [4]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm
import zipfile
import hashlib


class DLProgress(tqdm):
    """
    Handle Progress Bar while Downloading
    """
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        """
        A hook function that will be called once on establishment of the network connection and
        once after each block read thereafter.
        :param block_num: A count of blocks transferred so far
        :param block_size: Block size in bytes
        :param total_size: The total size of the file. This may be -1 on older FTP servers which do not return
                            a file size in response to a retrieval request.
        """
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

def _unzip(save_path, _, database_name, data_path):
    """
    Unzip wrapper with the same interface as _ungzip
    :param save_path: The path of the gzip files
    :param database_name: Name of database
    :param data_path: Path to extract to
    :param _: HACK - Used to have to same interface as _ungzip
    """
    print('Extracting {}...'.format(database_name))
    with zipfile.ZipFile(save_path) as zf:
        zf.extractall(data_path)

def download_extract(database_name, data_path):
    """
    Download and extract database
    :param database_name: Database name
    """
    DATASET_ML1M = database_name

    if database_name == DATASET_ML1M:
        url = 'http://files.grouplens.org/datasets/movielens/ml-20m.zip'
        hash_code = 'cd245b17a1ae2cc31bb14903e1204af3'
        extract_path = os.path.join(data_path, database_name)
        save_path = os.path.join(data_path, database_name + '.zip')
        extract_fn = _unzip

    if os.path.exists(extract_path):
        print('Found {} Data'.format(database_name))
        return

    if not os.path.exists(data_path):
        os.makedirs(data_path)

    if not os.path.exists(save_path):
        with DLProgress(unit='B', unit_scale=True, miniters=1, desc='Downloading {}'.format(database_name)) as pbar:
            urlretrieve(
                url,
                save_path,
                pbar.hook)

    assert hashlib.md5(open(save_path, 'rb').read()).hexdigest() == hash_code, \
        '{} file is corrupted.  Remove the file and try again.'.format(save_path)

    os.makedirs(extract_path)
    try:
        extract_fn(save_path, extract_path, database_name, data_path)
    except Exception as err:
        shutil.rmtree(extract_path)  # Remove extraction folder if there is an error
        raise err

    print('Done.')
    # Remove compressed data
    os.remove(save_path)

In [5]:
data_dir = './'
download_extract('ml-20m', data_dir)

Extracting ml-20m...
Done.


## 数据来源

[MovieLens ml-20m](https://grouplens.org/datasets/movielens/20m/)

MovieLens 20M数据集

由138,000名用户向27,000部电影应用了2000万个评级和465,000个标签应用程序。包括标签基因组数据，在1,100个标签上有1200万个相关性分数。